In [12]:
import kagglehub
path = kagglehub.dataset_download("tongpython/cat-and-dog")
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1


In [13]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1'):
 for filename in filenames[:10]:
   print(os.path.join(dirname, filename))

/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/training_set/training_set/cats/cat.1193.jpg
/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/training_set/training_set/cats/cat.1646.jpg
/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/training_set/training_set/cats/cat.1729.jpg
/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/training_set/training_set/cats/cat.1084.jpg
/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/training_set/training_set/cats/cat.1021.jpg
/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/training_set/training_set/cats/cat.2726.jpg
/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/training_set/training_set/cats/cat.3593.jpg
/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/training_set/training_set/cats/cat.205.jpg
/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/training_set/training_set/cats/cat.3740.jpg
/root/.cach

Importing required libraries

In [14]:
import cv2
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

Parameters for HOG

In [15]:
CELL_SIZE = 8
BLOCK_SIZE = 2
NUM_BINS = 9
EPSILON = 1e-5

Compute Roberts cross gradients for an image

In [16]:
def compute_roberts(image):
    gx = image[:-1, :-1].astype(np.float32) - image[1:, 1:].astype(np.float32)
    gy = image[:-1, 1:].astype(np.float32) - image[1:, :-1].astype(np.float32)
    return gx, gy

Compute HOG features using Roberts cross gradients

In [17]:
def compute_hog(image):
    gx, gy = compute_roberts(image)

    # Compute magnitude and orientation
    magnitude = np.sqrt(gx**2 + gy**2)
    orientation = np.arctan2(gy, gx) * (180 / np.pi)
    orientation = np.mod(orientation, 180)

    # Trim to multiples of CELL_SIZE
    h, w = magnitude.shape
    num_cells_y, num_cells_x = h // CELL_SIZE, w // CELL_SIZE
    magnitude = magnitude[:num_cells_y * CELL_SIZE, :num_cells_x * CELL_SIZE]
    orientation = orientation[:num_cells_y * CELL_SIZE, :num_cells_x * CELL_SIZE]

    # Reshape into cells
    cell_mag = magnitude.reshape(num_cells_y, CELL_SIZE, num_cells_x, CELL_SIZE).transpose(0, 2, 1, 3)
    cell_ori = orientation.reshape(num_cells_y, CELL_SIZE, num_cells_x, CELL_SIZE).transpose(0, 2, 1, 3)

    # Compute cell histograms
    cell_hist = np.zeros((num_cells_y, num_cells_x, NUM_BINS))
    bin_width = 180 / NUM_BINS

    for y in range(num_cells_y):
        for x in range(num_cells_x):
            mag_cell = cell_mag[y, x]
            ori_cell = cell_ori[y, x]
            hist = np.zeros(NUM_BINS)

            for i in range(CELL_SIZE):
                for j in range(CELL_SIZE):
                    theta = ori_cell[i, j]
                    mag = mag_cell[i, j]
                    bin_idx = theta / bin_width
                    lower_bin = int(np.floor(bin_idx)) % NUM_BINS
                    upper_bin = (lower_bin + 1) % NUM_BINS
                    fraction = bin_idx - np.floor(bin_idx)

                    hist[lower_bin] += mag * (1 - fraction)
                    hist[upper_bin] += mag * fraction
            cell_hist[y, x] = hist

    # Block normalization
    features = []
    for y in range(num_cells_y - BLOCK_SIZE + 1):
        for x in range(num_cells_x - BLOCK_SIZE + 1):
            block = cell_hist[y:y + BLOCK_SIZE, x:x + BLOCK_SIZE].ravel()

            # L2 normalization
            norm = np.sqrt(np.sum(block**2) + EPSILON)
            block_norm = block / norm

            # Clip and re-normalize
            block_norm = np.clip(block_norm, 0, 0.2)
            norm_clip = np.sqrt(np.sum(block_norm**2) + EPSILON)
            block_norm = block_norm / norm_clip

            features.extend(block_norm)

    return np.array(features)

Load and preprocess an image

In [18]:
def load_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Unable to load {image_path}")
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (64, 64))
    return resized

Load dataset and compute HOG features

In [19]:
def load_dataset(dataset_dir):
    features, labels = [], []

    for class_name in ['cats', 'dogs']:
        class_dir = os.path.join(dataset_dir, class_name)
        label = 0 if class_name == 'cats' else 1

        for filename in tqdm(os.listdir(class_dir), desc=f"Processing {class_name}"):
            if filename.endswith('.jpg'):
                image_path = os.path.join(class_dir, filename)
                image = load_image(image_path)
                if image is None:
                    continue
                hog = compute_hog(image)
                features.append(hog)
                labels.append(label)

    return np.vstack(features), np.array(labels)

Load datasets

In [20]:
train_dir = '/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/training_set/training_set/'
test_dir = '/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/test_set/test_set/'

In [21]:
X_train, y_train = load_dataset(train_dir)
X_test, y_test = load_dataset(test_dir)

Processing dogs: 100%|██████████| 1013/1013 [00:23<00:00, 44.04it/s]


Train Random Forest classifier

In [22]:
clf = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

Evaluate

In [23]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.7079
